# Data Processing using Pyspark

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

In [ ]:
# Load csv Dataset
#df=spark.read.csv('s3://<bucket/dir>/sample_data.csv',inferSchema=True,header=True)
df=spark.read.csv("gdrive/MyDrive/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv",inferSchema=True,header=True)

In [ ]:
#columns of dataframe
df.columns

In [ ]:
#check number of columns
len(df.columns)

In [ ]:
#number of records in dataframe
df.count()

In [ ]:
#shape of dataset
print((df.count(),len(df.columns)))

In [ ]:
#printSchema
df.printSchema()

In [ ]:
#fisrt few rows of dataframe
df.show(5)

In [ ]:

#select only 2 columns
df.select('Nombre departamento','Nombre municipio').show(5)

In [ ]:
#info about dataframe
df.describe().show()

In [ ]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

In [ ]:
#with column
df.withColumn("Edad_after_10_yrs",(df["Edad"]+10)).show(10,False)

In [ ]:
df.withColumn('Edad_double',df['Edad'].cast(DoubleType())).show(10,False)

In [ ]:
#with column
df.withColumn("Edad_after_10_yrs",(df["Edad"]+10)).show(10,False)

In [ ]:
#filter the records
df.filter(df['Tipo de contagio']=='Importado').show()

In [ ]:
#filter the records
df.filter(df['Tipo de contagio']=='Importado').select('Edad','Sexo','Tipo de contagio').show()

In [ ]:
#filter the multiple conditions
df.filter(df['Tipo de contagio']=='Importado').filter(df['ID de caso'] >10).show()

In [ ]:
#filter the multiple conditions
df.filter((df['Tipo de contagio']=='Importado')&(df['ID de caso'] >10)).show()

In [ ]:
#Distinct Values in a column
df.select('Tipo de contagio').distinct().show()

In [ ]:
#distinct value count
df.select('Tipo de contagio').distinct().count()

In [ ]:
df.groupBy('Tipo de contagio').count().show(5,False)

In [ ]:
# Value counts
df.groupBy('Tipo de contagio').count().orderBy('count',ascending=False).show(5,False)

In [ ]:
# Value counts
df.groupBy('Tipo de contagio').mean().show(5,False)

In [ ]:
df.groupBy('Tipo de contagio').sum().show(5,False)

In [ ]:
# Value counts
df.groupBy('Tipo de contagio').max().show(5,False)

In [ ]:
# Value counts
df.groupBy('Tipo de contagio').min().show(5,False)

In [ ]:
#Aggregation
df.groupBy('Tipo de contagio').agg({'ID de caso':'sum'}).show(5,False)

In [ ]:
# UDF
from pyspark.sql.functions import udf


In [ ]:
#normal function
def price_range(brand):
    if brand in ['Comunitaria', 'Relacionado']:
        return 'Comunitaria o Relacionado'
    elif brand =='Importado':
        return 'Importado'
    else:
        return 'En estudio'

In [ ]:
#create udf using python function
brand_udf=udf(price_range,StringType())
#apply udf on dataframe
df.withColumn('price_range',brand_udf(df['Tipo de contagio'])).show(10,False)

In [ ]:
#using lambda function
age_udf = udf(lambda Edad: "young" if Edad <= 30 else "senior", StringType())
#apply udf on dataframe
df.withColumn("age_group", age_udf(df.Edad)).show(10,False)

In [ ]:
#pandas udf
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [ ]:
#create python function
def remaining_yrs(age):
    yrs_left=100-age

    return yrs_left

In [ ]:
#create udf using python function
length_udf = udf(remaining_yrs, IntegerType())
#apply pandas udf on dataframe
df.withColumn("yrs_left", length_udf(df['Edad'])).show(10,False)

In [ ]:
#udf using two columns
def prod(rating,exp):
    x=rating*exp
    return x

In [ ]:
#create udf using python function
prod_udf = udf(prod, DoubleType())
#apply pandas udf on multiple columns of dataframe
df.withColumn("product", prod_udf(df['Edad'],df['ID de caso'])).show(10,False)

In [ ]:
#duplicate values
df.count()

In [ ]:
#drop duplicate values
df=df.dropDuplicates()

In [ ]:
#validate new count
df.count()

In [ ]:
#drop column of dataframe
df_new=df.drop('Tipo de contagio')

In [ ]:
df_new.show(10)

In [ ]:
# saving file (csv)

In [ ]:
#target directory
write_uri='s3://jupyterjorge/trusted/datasets/covid19/results'

In [ ]:
#save the dataframe as single csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

In [ ]:
# parquet

In [ ]:
#target location
parquet_uri='s3://jupyterjorge/trusted/datasets/covid19/results_parquet'

In [ ]:
#save the data into parquet format
df.write.format('parquet').save(parquet_uri)